In [214]:
#Bibliotecas
import librosa
import itertools
import numpy as np
import pandas as pd
from os import listdir
from os import chdir
from sklearn.cluster import KMeans  
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [215]:
#Semente
np.random.seed(seed = 42)

In [216]:
def SubSegmentacao(audio,fs,tamanho,tempo,pontos):
    i = pontos
    #audio_h = librosa.effects.harmonic(audio)
    D = np.abs(librosa.stft(audio))**2
    S = librosa.feature.melspectrogram(S=D)
    db = librosa.amplitude_to_db(S)
    while True:
        try:
            bounds = librosa.segment.agglomerative(db, i)
            break
        except:
            i = i - 1
    bound_times = librosa.frames_to_time(bounds, sr=fs)
    inicio = int(tamanho*(np.min(bound_times[1:])/tempo))
    fim    = int(tamanho*(np.max(bound_times[1:])/tempo))
    return (audio[inicio:fim])

In [217]:
def SegmentarAudio(audio,fs,n,tamanho,tempo):
    i = 500
    #audio_h = librosa.effects.harmonic(audio)
    D = np.abs(librosa.stft(audio))**2
    S = librosa.feature.melspectrogram(S=D)
    db = librosa.amplitude_to_db(S)
    while True:
        try:
            bounds = librosa.segment.agglomerative(db, i)
            break
        except:
            i = i - 1
    bound_times = librosa.frames_to_time(bounds, sr=fs)
    bound_times = np.vstack((bound_times[2:], [0]*len(bound_times[2:]))).T
    kmeans = KMeans(n_clusters=n)  
    kmeans.fit(bound_times[2:])  
    func1 = lambda x: int(tamanho*(x/tempo))
    centros =  np.array([func1(xi) for xi in kmeans.cluster_centers_[:,0]])
    centros =  np.append(centros,tamanho)
    centros.sort()
    ultimo  = 0
    retorno = []
    for i,j in zip(centros,centros[1:]):
        sl = int((i+j)/2)
        retorno.append(SubSegmentacao(audio[ultimo:sl],fs,tamanho,tempo,10))
        ultimo = sl
    return np.array(retorno)

In [218]:
scaler      = StandardScaler()
nLetras     = 4   #Número de letras que constará por arquivo
n_mfcc      = 45 #Número de componentes MFCC que irá ser extraido

#Pasta = Caminho da pasta que contém os arquivos de treinamento
def TreinarModelo(Pasta):
    chdir(Pasta)
    print('Extraindo atributo MFCC dos arquivos...')
    atributos  = []
    rotulos    = []
    arquivos = listdir()
    arquivos.sort()
    for arquivo in arquivos:
        if 'wav' in arquivo:
            audio, fs = librosa.load(arquivo,None)
            tamanho = audio.shape[0]
            tempo   = audio.shape[0]/fs
            retorno = SegmentarAudio(audio, fs, nLetras, tamanho, tempo)
            print('Extraindo MFCC do arquivo ', arquivo, '.')
            for i in range(0, nLetras): 
                retorno[i,] = librosa.util.normalize(retorno[i,],axis=0, norm=1)
                y_harm, y_per = librosa.effects.hpss(retorno[i,])
                
                #mfcc2    =  librosa.feature.mfcc(y_harm,  sr=fs, n_mfcc=n_mfcc,hop_length=128).T
                mfcc2    =  librosa.feature.mfcc(retorno[i,],  sr=fs, n_mfcc=n_mfcc,hop_length=64).T
                rec = librosa.segment.recurrence_matrix(mfcc2, mode='affinity',metric='jaccard', sparse=True)
                mfcc2 = librosa.decompose.nn_filter(mfcc2, rec=rec,aggregate=np.average)
                #mfcc2 = librosa.decompose.nn_filter(mfcc2,aggregate=np.average,metric='manhattan')
                
                #mfcc1    =  librosa.feature.mfcc(retorno[i,],  sr=fs, n_mfcc=n_mfcc,hop_length=100).T
                #rec = librosa.segment.recurrence_matrix(mfcc1, mode='affinity',metric='cosine', sparse=True)
               # mfcc1 = librosa.decompose.nn_filter(mfcc1, rec=rec,aggregate=np.average)
                
                #valor    =  np.concatenate((mfcc1,mfcc2))
                valor    = scaler.fit_transform(mfcc2)
                atributos.append(valor)
                rotulos = rotulos+[(arquivo[i])]*len(valor)
    print('Treinando o modelo...')
    atributos = np.vstack(atributos)
    rotulos   = np.array(rotulos)
    modelo = RandomForestClassifier(n_estimators=15,criterion="entropy",n_jobs=-1)
    modelo.fit(atributos, rotulos)
    return modelo

In [219]:
chdir('..')
modelo = TreinarModelo('base_treinamento')
print('Modelo treinado!')

Extraindo atributo MFCC dos arquivos...
Extraindo MFCC do arquivo  67aa.wav .


/home/pparreira/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/pparreira/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Extraindo MFCC do arquivo  67mc.wav .


/home/pparreira/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/pparreira/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Extraindo MFCC do arquivo  67n6.wav .


...

Extraindo MFCC do arquivo  xxca.wav .


/home/pparreira/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/pparreira/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Extraindo MFCC do arquivo  xxna.wav .


/home/pparreira/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/pparreira/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Treinando o modelo...
Modelo treinado!


In [220]:
def ValidacaoModelo(Pasta, Modelo):
    lstPreditos = []
    lstReais    = []
    nAcertos    = 0
    nAcertoCaracter = 0
    nQuantidade = 0
    nQuantidadeCaracter = 0
    chdir(Pasta)
    arquivos = listdir()
    arquivos.sort()
    print("Iniciando a Validação...")
    for arquivo in arquivos:
        if 'wav' in arquivo: 
            audio, fs = librosa.load(arquivo,None)
            tamanho = audio.shape[0]
            tempo   = audio.shape[0]/fs
            retorno = SegmentarAudio(audio, fs, nLetras, tamanho, tempo)
            nAcerto = 0
            nQuantidade = nQuantidade + 1
            nQuantidadeCaracter = nQuantidadeCaracter + 4
            print('Validando o arquivo ', arquivo, '.')
            for i in range(0,nLetras):
                retorno[i,] = librosa.util.normalize(retorno[i,],axis=0, norm=1)
                y_harm, y_per = librosa.effects.hpss(retorno[i,])
                
                #mfcc2    =  librosa.feature.mfcc(y_harm,  sr=fs, n_mfcc=n_mfcc,hop_length=128).T
                mfcc2    =  librosa.feature.mfcc(retorno[i,],  sr=fs, n_mfcc=n_mfcc,hop_length=64).T
                rec = librosa.segment.recurrence_matrix(mfcc2, mode='affinity',metric='jaccard', sparse=True)
                mfcc2 = librosa.decompose.nn_filter(mfcc2, rec=rec,aggregate=np.average)
                #mfcc2 = librosa.decompose.nn_filter(mfcc2,aggregate=np.average,metric='manhattan')
                #mfcc1    =  librosa.feature.mfcc(retorno[i,],  sr=fs, n_mfcc=n_mfcc,hop_length=100).T
                #rec = librosa.segment.recurrence_matrix(mfcc1, mode='affinity',metric='cosine', sparse=True)
               # mfcc1 = librosa.decompose.nn_filter(mfcc1, rec=rec,aggregate=np.average)
                
                #valor    =  np.concatenate((mfcc1,mfcc2))
                valor    = scaler.fit_transform(mfcc2)
                pred     = Modelo.predict(valor) 
                (values,counts) = np.unique(pred,return_counts=True)
                ind = np.argmax(counts)
                lstPreditos.append(values[ind])
                lstReais.append(arquivo[i])
                if values[ind] == arquivo[i]:
                    nAcerto = nAcerto + 1
                    nAcertoCaracter = nAcertoCaracter + 1
        if nAcerto == 4:
            nAcertos = nAcertos + 1
        else:
            print("errou!")
    
    print("Validação Encerrada!")
    return lstPreditos, lstReais, (nAcertos/nQuantidade), (nAcertoCaracter/nQuantidadeCaracter)

In [221]:
lstPreditos = []
lstReais    = []
vAcuraciaCAPTCHA   = 0.0
vAcuraciaCaracter   = 0.0

chdir('..')
lstPreditos, lstReais, vAcuraciaCAPTCHA, vAcuraciaCaracter  = ValidacaoModelo('base_validacao',modelo)

Iniciando a Validação...
Validando o arquivo  66dn.wav .


/home/pparreira/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/pparreira/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


errou!
Validando o arquivo  6ah7.wav .


/home/pparreira/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/pparreira/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


errou!
Validando o arquivo  6amx.wav .


...

errou!
Validando o arquivo  xxbc.wav .


/home/pparreira/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/pparreira/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


errou!
Validando o arquivo  xxhc.wav .


/home/pparreira/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/pparreira/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


errou!
Validação Encerrada!


In [222]:
print("Acurácia por CAPTCHA: {:.2f}".format(vAcuraciaCAPTCHA*100),"%")

print("Acurácia por Caracter: {:.2f}".format(vAcuraciaCaracter*100),"%")

Acurácia por CAPTCHA: 25.50 %
Acurácia por Caracter: 67.11 %


In [223]:
Areais    =  np.array(lstReais)
Apreditos =  np.array(lstPreditos)
print ("Matriz de Confusão\n")
print (pd.crosstab(Areais,Apreditos,rownames=['REAL'],colnames=['PREDITO'],margins=True))

Matriz de Confusão

PREDITO   6   7   a   b   c   d   h   m   n   x  All
REAL                                                
6        56   3   0   3   3   0   0   2   3   0   70
7         5  55   1   0   0   0   0   0   2   0   63
a         0   1  54   0   0   0   8   0   1   0   64
b         0   1   1  36   1   2   0   1  11   0   53
c        28   3   2   2  27   0   0   1   2   1   66
d         0   1   3  27   0  10   0   2  12   0   55
h         1   0   2   0   0   0  53   0   0   0   56
m         1   0   0   4   0   0   1  22  29   0   57
n         0   1   0   9   0   0   0   7  28   0   45
x         3   1   0   1   2   0   0   1   0  59   67
All      94  66  63  82  33  12  62  36  88  60  596
